In [ ]:
###########################################################
# Imports
###########################################################
import sys
import os
sys.path.insert(0, os.getcwd()[0:len(os.getcwd()) - 8])

import uproot
import numpy as np
import math
import torch

import Padding
import Normalise

In [ ]:
MAX_EVENTS = 10000

In [ ]:
###########################################################
# Define file
###########################################################

#fileName = sys.path[0] + '/files/hierarchy_VALIDATION.root'
fileName = sys.path[0] + '/files/MLPHierarchyTrainingFile.root'
outputFile = sys.path[0] + '/files/networkScores.npz'

treeFile = uproot.open(fileName)

In [ ]:
# Model paths
PRIMARY_TRACK_BRANCH_MODEL_PATH = sys.path[0] + '/models/primary_track_branch_model'
TRACK_TRACK_BRANCH_MODEL_PATH = sys.path[0] + '/models/track_track_branch_model'
TRACK_SHOWER_BRANCH_MODEL_PATH = sys.path[0] + '/models/track_shower_branch_model'

PRIMARY_TRACK_CLASSIFIER_MODEL_PATH = sys.path[0] + '/models/primary_track_classifier_model'
PRIMARY_SHOWER_CLASSIFIER_MODEL_PATH = sys.path[0] + '/models/primary_shower_classifier_model'
TRACK_TRACK_CLASSIFIER_MODEL_PATH = sys.path[0] + '/models/track_track_classifier_model'
TRACK_SHOWER_CLASSIFIER_MODEL_PATH = sys.path[0] + '/models/track_shower_classifier_model'

# Models
primary_track_branch_model = torch.jit.load(PRIMARY_TRACK_BRANCH_MODEL_PATH)
track_track_branch_model = torch.jit.load(TRACK_TRACK_BRANCH_MODEL_PATH)
track_shower_branch_model = torch.jit.load(TRACK_SHOWER_BRANCH_MODEL_PATH)

primary_track_classifier_model = torch.jit.load(PRIMARY_TRACK_CLASSIFIER_MODEL_PATH)
primary_shower_classifier_model = torch.jit.load(PRIMARY_SHOWER_CLASSIFIER_MODEL_PATH)
track_track_classifier_model = torch.jit.load(TRACK_TRACK_CLASSIFIER_MODEL_PATH)
track_shower_classifier_model = torch.jit.load(TRACK_SHOWER_CLASSIFIER_MODEL_PATH)

In [ ]:
######################
# Read Trees
######################       
# Event 
eventTree = treeFile['EventTree']
eventBranches = eventTree.arrays()
event_eventID = np.array(eventBranches["EventID"])
event_nRecoParticles = np.array(eventBranches["NRecoParticles"])
event_nPrimaryTrackLinks = np.array(eventBranches["NPrimaryTrackLinks"])
event_nPrimaryShowerLinks = np.array(eventBranches["NPrimaryShowerLinks"])
event_nLaterTierTrackTrackLinks = np.array(eventBranches["NLaterTierTrackTrackLinks"])
event_nLaterTierTrackShowerLinks = np.array(eventBranches["NLaterTierTrackShowerLinks"])

# Primary track   
primaryTrackTree = treeFile['PrimaryTrackTree']
primaryTrackBranches = primaryTrackTree.arrays()
primaryTrack_particleID = np.array(primaryTrackBranches["ParticleID"])
primaryTrack_nSpacepoints = np.array(primaryTrackBranches['NSpacepoints'])
primaryTrack_nuVertexSeparation = np.array(primaryTrackBranches['NuSeparation'])
primaryTrack_startRegionNHits = np.array(primaryTrackBranches['VertexRegionNHits'])            
primaryTrack_startRegionNParticles = np.array(primaryTrackBranches['VertexRegionNParticles'])            
primaryTrack_dca = np.array(primaryTrackBranches['DCA'])            
primaryTrack_connectionExtrapDistance = np.array(primaryTrackBranches['ConnectionExtrapDistance'])
primaryTrack_isPOIClosestToNu = np.array(primaryTrackBranches['IsPOIClosestToNu'])
primaryTrack_parentConnectionDistance = np.array(primaryTrackBranches['ParentConnectionDistance'])
primaryTrack_childConnectionDistance = np.array(primaryTrackBranches['ChildConnectionDistance']) 
primaryTrack_trueVisibleGeneration = np.array(primaryTrackBranches["TrueVisibleGeneration"])
primaryTrack_trueVisibleParentID = np.array(primaryTrackBranches['TrueVisibleParentID'])
primaryTrack_upstreamVertexX = np.array(primaryTrackBranches['UpstreamVertexX'])
primaryTrack_upstreamVertexY = np.array(primaryTrackBranches['UpstreamVertexY'])
primaryTrack_upstreamVertexZ = np.array(primaryTrackBranches['UpstreamVertexZ'])                                      
primaryTrack_downstreamVertexX = np.array(primaryTrackBranches['DownstreamVertexX'])
primaryTrack_downstreamVertexY = np.array(primaryTrackBranches['DownstreamVertexY'])
primaryTrack_downstreamVertexZ = np.array(primaryTrackBranches['DownstreamVertexZ'])                                        

# Primary shower   
primaryShowerTree = treeFile['PrimaryShowerTree']
primaryShowerBranches = primaryShowerTree.arrays()
primaryShower_particleID = np.array(primaryShowerBranches["ParticleID"])
primaryShower_nSpacepoints = np.array(primaryShowerBranches['NSpacepoints'])
primaryShower_nuVertexSeparation = np.array(primaryShowerBranches['NuSeparation'])
primaryShower_startRegionNHits = np.array(primaryShowerBranches['VertexRegionNHits'])            
primaryShower_startRegionNParticles = np.array(primaryShowerBranches['VertexRegionNParticles'])            
primaryShower_dca = np.array(primaryShowerBranches['DCA'])            
primaryShower_connectionExtrapDistance = np.array(primaryShowerBranches['ConnectionExtrapDistance'])
primaryShower_isPOIClosestToNu = np.array(primaryShowerBranches['IsPOIClosestToNu'])
primaryShower_parentConnectionDistance = np.array(primaryShowerBranches['ParentConnectionDistance'])
primaryShower_childConnectionDistance = np.array(primaryShowerBranches['ChildConnectionDistance']) 
primaryShower_trueVisibleGeneration = np.array(primaryShowerBranches["TrueVisibleGeneration"])
primaryShower_trueVisibleParentID = np.array(primaryShowerBranches['TrueVisibleParentID'])
primaryShower_upstreamVertexX = np.array(primaryShowerBranches['UpstreamVertexX'])
primaryShower_upstreamVertexY = np.array(primaryShowerBranches['UpstreamVertexY'])
primaryShower_upstreamVertexZ = np.array(primaryShowerBranches['UpstreamVertexZ'])                                      
primaryShower_downstreamVertexX = np.array(primaryShowerBranches['DownstreamVertexX'])
primaryShower_downstreamVertexY = np.array(primaryShowerBranches['DownstreamVertexY'])
primaryShower_downstreamVertexZ = np.array(primaryShowerBranches['DownstreamVertexZ']) 

# track-track
trackTrackTree = treeFile['LaterTierTrackTrackTree']
trackTrackBranches = trackTrackTree.arrays()
trackTrack_childID = np.array(trackTrackBranches["ChildParticleID"])
trackTrack_parentID = np.array(trackTrackBranches["ParentParticleID"])
trackTrack_parentTrackScore = np.array(trackTrackBranches['ParentTrackScore'])            
trackTrack_childTrackScore = np.array(trackTrackBranches['ChildTrackScore'])
trackTrack_parentNSpacepoints = np.array(trackTrackBranches['ParentNSpacepoints'])
trackTrack_parentNSpacepoints_notNorm = np.array(trackTrackBranches['ParentNSpacepoints'])
trackTrack_childNSpacepoints = np.array(trackTrackBranches['ChildNSpacepoints'])
trackTrack_separation3D = np.array(trackTrackBranches['Separation3D'])
trackTrack_separation3D_notNorm = np.array(trackTrackBranches['Separation3D'])
trackTrack_parentNuVertexSep = np.array(trackTrackBranches['ParentNuVertexSep'])
trackTrack_childNuVertexSep = np.array(trackTrackBranches['ChildNuVertexSep'])
trackTrack_childNuVertexSep_notNorm = np.array(trackTrackBranches['ChildNuVertexSep'])
trackTrack_parentEndRegionNHits = np.array(trackTrackBranches['ParentEndRegionNHits'])
trackTrack_parentEndRegionNParticles = np.array(trackTrackBranches['ParentEndRegionNParticles'])
trackTrack_parentEndRegionRToWall = np.array(trackTrackBranches['ParentEndRegionRToWall'])
trackTrack_vertexSeparation = np.array(trackTrackBranches['VertexSeparation'])        
trackTrack_doesChildConnect = np.array(trackTrackBranches['DoesChildConnect'])
trackTrack_overshootStartDCA = np.array(trackTrackBranches['OvershootStartDCA'])
trackTrack_overshootStartL = np.array(trackTrackBranches['OvershootStartL'])
trackTrack_overshootEndDCA = np.array(trackTrackBranches['OvershootEndDCA'])
trackTrack_overshootEndL = np.array(trackTrackBranches['OvershootEndL'])
trackTrack_childConnectionDCA = np.array(trackTrackBranches['ChildCPDCA'])
trackTrack_childConnectionExtrapDistance = np.array(trackTrackBranches['ChildCPExtrapDistance'])
trackTrack_childConnectionLRatio = np.array(trackTrackBranches['ChildCPLRatio'])
trackTrack_parentConnectionPointNUpstreamHits = np.array(trackTrackBranches['ParentCPNUpstreamHits'])
trackTrack_parentConnectionPointNDownstreamHits = np.array(trackTrackBranches['ParentCPNDownstreamHits'])
trackTrack_parentConnectionPointNHitRatio = np.array(trackTrackBranches['ParentCPNHitRatio'])
trackTrack_parentConnectionPointEigenValueRatio = np.array(trackTrackBranches['ParentCPEigenvalueRatio'])
trackTrack_parentConnectionPointOpeningAngle = np.array(trackTrackBranches['ParentCPOpeningAngle'])
trackTrack_isParentPOIClosestToNu = np.array(trackTrackBranches['ParentIsPOIClosestToNu'])
trackTrack_isChildPOIClosestToNu = np.array(trackTrackBranches['ChildIsPOIClosestToNu'])
trackTrack_trueChildGen = np.array(trackTrackBranches['ChildTrueVisibleGeneration'])

# track-shower
trackShowerTree = treeFile['LaterTierTrackShowerTree']
trackShowerBranches = trackShowerTree.arrays()
trackShower_childID = np.array(trackShowerBranches["ChildParticleID"])
trackShower_parentID = np.array(trackShowerBranches["ParentParticleID"])
trackShower_parentTrackScore = np.array(trackShowerBranches['ParentTrackScore'])            
trackShower_childTrackScore = np.array(trackShowerBranches['ChildTrackScore'])
trackShower_parentNSpacepoints = np.array(trackShowerBranches['ParentNSpacepoints'])
trackShower_parentNSpacepoints_notNorm = np.array(trackShowerBranches['ParentNSpacepoints'])
trackShower_childNSpacepoints = np.array(trackShowerBranches['ChildNSpacepoints'])
trackShower_separation3D = np.array(trackShowerBranches['Separation3D'])
trackShower_separation3D_notNorm = np.array(trackShowerBranches['Separation3D'])
trackShower_parentNuVertexSep = np.array(trackShowerBranches['ParentNuVertexSep'])
trackShower_childNuVertexSep = np.array(trackShowerBranches['ChildNuVertexSep'])     
trackShower_childNuVertexSep_notNorm = np.array(trackShowerBranches['ChildNuVertexSep'])
trackShower_parentEndRegionNHits = np.array(trackShowerBranches['ParentEndRegionNHits'])
trackShower_parentEndRegionNParticles = np.array(trackShowerBranches['ParentEndRegionNParticles'])
trackShower_parentEndRegionRToWall = np.array(trackShowerBranches['ParentEndRegionRToWall'])
trackShower_vertexSeparation = np.array(trackShowerBranches['VertexSeparation'])        
trackShower_doesChildConnect = np.array(trackShowerBranches['DoesChildConnect'])
trackShower_overshootStartDCA = np.array(trackShowerBranches['OvershootStartDCA'])
trackShower_overshootStartL = np.array(trackShowerBranches['OvershootStartL'])
trackShower_overshootEndDCA = np.array(trackShowerBranches['OvershootEndDCA'])
trackShower_overshootEndL = np.array(trackShowerBranches['OvershootEndL'])
trackShower_childConnectionDCA = np.array(trackShowerBranches['ChildCPDCA'])
trackShower_childConnectionExtrapDistance = np.array(trackShowerBranches['ChildCPExtrapDistance'])
trackShower_childConnectionLRatio = np.array(trackShowerBranches['ChildCPLRatio'])
trackShower_parentConnectionPointNUpstreamHits = np.array(trackShowerBranches['ParentCPNUpstreamHits'])
trackShower_parentConnectionPointNDownstreamHits = np.array(trackShowerBranches['ParentCPNDownstreamHits'])
trackShower_parentConnectionPointNHitRatio = np.array(trackShowerBranches['ParentCPNHitRatio'])
trackShower_parentConnectionPointEigenValueRatio = np.array(trackShowerBranches['ParentCPEigenvalueRatio'])
trackShower_parentConnectionPointOpeningAngle = np.array(trackShowerBranches['ParentCPOpeningAngle'])
trackShower_isParentPOIClosestToNu = np.array(trackShowerBranches['ParentIsPOIClosestToNu'])
trackShower_isChildPOIClosestToNu = np.array(trackShowerBranches['ChildIsPOIClosestToNu'])
trackShower_trueChildGen = np.array(trackShowerBranches['ChildTrueVisibleGeneration'])                                                                        

In [ ]:
#############################
# Normalise
#############################
# primary track
Normalise.normaliseXAxis(primaryTrack_nSpacepoints, Normalise.primaryNSpacepoints_min, Normalise.primaryNSpacepoints_max)    
Normalise.normaliseXAxis(primaryTrack_nuVertexSeparation, Normalise.primaryNuVertexSeparation_min, Normalise.primaryNuVertexSeparation_max)    
Normalise.normaliseXAxis(primaryTrack_startRegionNHits, Normalise.primaryStartRegionNHits_min, Normalise.primaryStartRegionNHits_max)    
Normalise.normaliseXAxis(primaryTrack_startRegionNParticles, Normalise.primaryStartRegionNParticles_min, Normalise.primaryStartRegionNParticles_max)    
Normalise.normaliseXAxis(primaryTrack_dca, Normalise.primaryDCA_min, Normalise.primaryDCA_max)
Normalise.normaliseXAxis(primaryTrack_connectionExtrapDistance, Normalise.primaryConnectionExtrapDistance_min, Normalise.primaryConnectionExtrapDistance_max)
Normalise.normaliseXAxis(primaryTrack_parentConnectionDistance, Normalise.primaryClosestParentL_min, Normalise.primaryClosestParentL_max) 
Normalise.normaliseXAxis(primaryTrack_childConnectionDistance, Normalise.primaryClosestParentT_min, Normalise.primaryClosestParentT_max) 
# primary shower
Normalise.normaliseXAxis(primaryShower_nSpacepoints, Normalise.primaryNSpacepoints_min, Normalise.primaryNSpacepoints_max)    
Normalise.normaliseXAxis(primaryShower_nuVertexSeparation, Normalise.primaryNuVertexSeparation_min, Normalise.primaryNuVertexSeparation_max)    
Normalise.normaliseXAxis(primaryShower_startRegionNHits, Normalise.primaryStartRegionNHits_min, Normalise.primaryStartRegionNHits_max)    
Normalise.normaliseXAxis(primaryShower_startRegionNParticles, Normalise.primaryStartRegionNParticles_min, Normalise.primaryStartRegionNParticles_max)    
Normalise.normaliseXAxis(primaryShower_dca, Normalise.primaryDCA_min, Normalise.primaryDCA_max)
Normalise.normaliseXAxis(primaryShower_connectionExtrapDistance, Normalise.primaryConnectionExtrapDistance_min, Normalise.primaryConnectionExtrapDistance_max)
Normalise.normaliseXAxis(primaryShower_parentConnectionDistance, Normalise.primaryClosestParentL_min, Normalise.primaryClosestParentL_max) 
Normalise.normaliseXAxis(primaryShower_childConnectionDistance, Normalise.primaryClosestParentT_min, Normalise.primaryClosestParentT_max) 
# Later tier track-track
Normalise.normaliseXAxis(trackTrack_parentTrackScore, Normalise.parentTrackScore_min, Normalise.parentTrackScore_max)
Normalise.normaliseXAxis(trackTrack_childTrackScore, Normalise.parentTrackScore_min, Normalise.parentTrackScore_max)    
Normalise.normaliseXAxis(trackTrack_parentNSpacepoints, Normalise.parentNSpacepoints_min, Normalise.parentNSpacepoints_max)   
Normalise.normaliseXAxis(trackTrack_childNSpacepoints, Normalise.parentNSpacepoints_min, Normalise.parentNSpacepoints_max)        
Normalise.normaliseXAxis(trackTrack_separation3D, Normalise.separation3D_min, Normalise.separation3D_max)     
Normalise.normaliseXAxis(trackTrack_parentNuVertexSep, Normalise.parentNuVertexSeparation_min, Normalise.parentNuVertexSeparation_max) 
Normalise.normaliseXAxis(trackTrack_childNuVertexSep, Normalise.parentNuVertexSeparation_min, Normalise.parentNuVertexSeparation_max) 
Normalise.normaliseXAxis(trackTrack_parentEndRegionNHits, Normalise.parentEndRegionNHits_min, Normalise.parentEndRegionNHits_max)
Normalise.normaliseXAxis(trackTrack_parentEndRegionNParticles, Normalise.parentEndRegionNParticles_min, Normalise.parentEndRegionNParticles_max)
Normalise.normaliseXAxis(trackTrack_parentEndRegionRToWall, Normalise.parentEndRegionRToWall_min, Normalise.parentEndRegionRToWall_max)
Normalise.normaliseXAxis(trackTrack_vertexSeparation, Normalise.vertexSeparation_min, Normalise.vertexSeparation_max)
Normalise.normaliseXAxis(trackTrack_doesChildConnect, Normalise.doesChildConnect_min, Normalise.doesChildConnect_max)
Normalise.normaliseXAxis(trackTrack_overshootStartDCA, Normalise.overshootDCA_min, Normalise.overshootDCA_max)
Normalise.normaliseXAxis(trackTrack_overshootStartL, Normalise.overshootL_min, Normalise.overshootL_max)
Normalise.normaliseXAxis(trackTrack_overshootEndDCA, Normalise.overshootDCA_min, Normalise.overshootDCA_max)
Normalise.normaliseXAxis(trackTrack_overshootEndL, Normalise.overshootL_min, Normalise.overshootL_max)
Normalise.normaliseXAxis(trackTrack_childConnectionDCA, Normalise.childConnectionDCA_min, Normalise.childConnectionDCA_max)
Normalise.normaliseXAxis(trackTrack_childConnectionExtrapDistance, Normalise.childConnectionExtrapDistance_min, Normalise.childConnectionExtrapDistance_max)
Normalise.normaliseXAxis(trackTrack_childConnectionLRatio, Normalise.childConnectionLRatio_min, Normalise.childConnectionLRatio_max)
Normalise.normaliseXAxis(trackTrack_parentConnectionPointNUpstreamHits, Normalise.parentConnectionPointNUpstreamHits_min, Normalise.parentConnectionPointNUpstreamHits_max)
Normalise.normaliseXAxis(trackTrack_parentConnectionPointNDownstreamHits, Normalise.parentConnectionPointNDownstreamHits_min, Normalise.parentConnectionPointNDownstreamHits_max)
Normalise.normaliseXAxis(trackTrack_parentConnectionPointNHitRatio, Normalise.parentConnectionPointNHitRatio_min, Normalise.parentConnectionPointNHitRatio_max)
Normalise.normaliseXAxis(trackTrack_parentConnectionPointEigenValueRatio, Normalise.parentConnectionPointEigenValueRatio_min, Normalise.parentConnectionPointEigenValueRatio_max)
Normalise.normaliseXAxis(trackTrack_parentConnectionPointOpeningAngle, Normalise.parentConnectionPointOpeningAngle_min, Normalise.parentConnectionPointOpeningAngle_max) 
# Later tier track-shower
Normalise.normaliseXAxis(trackShower_parentTrackScore, Normalise.parentTrackScore_min, Normalise.parentTrackScore_max)
Normalise.normaliseXAxis(trackShower_childTrackScore, Normalise.parentTrackScore_min, Normalise.parentTrackScore_max)    
Normalise.normaliseXAxis(trackShower_parentNSpacepoints, Normalise.parentNSpacepoints_min, Normalise.parentNSpacepoints_max)   
Normalise.normaliseXAxis(trackShower_childNSpacepoints, Normalise.parentNSpacepoints_min, Normalise.parentNSpacepoints_max)        
Normalise.normaliseXAxis(trackShower_separation3D, Normalise.separation3D_min, Normalise.separation3D_max)     
Normalise.normaliseXAxis(trackShower_parentNuVertexSep, Normalise.parentNuVertexSeparation_min, Normalise.parentNuVertexSeparation_max) 
Normalise.normaliseXAxis(trackShower_childNuVertexSep, Normalise.parentNuVertexSeparation_min, Normalise.parentNuVertexSeparation_max) 
Normalise.normaliseXAxis(trackShower_parentEndRegionNHits, Normalise.parentEndRegionNHits_min, Normalise.parentEndRegionNHits_max)
Normalise.normaliseXAxis(trackShower_parentEndRegionNParticles, Normalise.parentEndRegionNParticles_min, Normalise.parentEndRegionNParticles_max)
Normalise.normaliseXAxis(trackShower_parentEndRegionRToWall, Normalise.parentEndRegionRToWall_min, Normalise.parentEndRegionRToWall_max)
Normalise.normaliseXAxis(trackShower_vertexSeparation, Normalise.vertexSeparation_min, Normalise.vertexSeparation_max)
Normalise.normaliseXAxis(trackShower_doesChildConnect, Normalise.doesChildConnect_min, Normalise.doesChildConnect_max)
Normalise.normaliseXAxis(trackShower_overshootStartDCA, Normalise.overshootDCA_min, Normalise.overshootDCA_max)
Normalise.normaliseXAxis(trackShower_overshootStartL, Normalise.overshootL_min, Normalise.overshootL_max)
Normalise.normaliseXAxis(trackShower_overshootEndDCA, Normalise.overshootDCA_min, Normalise.overshootDCA_max)
Normalise.normaliseXAxis(trackShower_overshootEndL, Normalise.overshootL_min, Normalise.overshootL_max)
Normalise.normaliseXAxis(trackShower_childConnectionDCA, Normalise.childConnectionDCA_min, Normalise.childConnectionDCA_max)
Normalise.normaliseXAxis(trackShower_childConnectionExtrapDistance, Normalise.childConnectionExtrapDistance_min, Normalise.childConnectionExtrapDistance_max)
Normalise.normaliseXAxis(trackShower_childConnectionLRatio, Normalise.childConnectionLRatio_min, Normalise.childConnectionLRatio_max)
Normalise.normaliseXAxis(trackShower_parentConnectionPointNUpstreamHits, Normalise.parentConnectionPointNUpstreamHits_min, Normalise.parentConnectionPointNUpstreamHits_max)
Normalise.normaliseXAxis(trackShower_parentConnectionPointNDownstreamHits, Normalise.parentConnectionPointNDownstreamHits_min, Normalise.parentConnectionPointNDownstreamHits_max)
Normalise.normaliseXAxis(trackShower_parentConnectionPointNHitRatio, Normalise.parentConnectionPointNHitRatio_min, Normalise.parentConnectionPointNHitRatio_max)
Normalise.normaliseXAxis(trackShower_parentConnectionPointEigenValueRatio, Normalise.parentConnectionPointEigenValueRatio_min, Normalise.parentConnectionPointEigenValueRatio_max)
Normalise.normaliseXAxis(trackShower_parentConnectionPointOpeningAngle, Normalise.parentConnectionPointOpeningAngle_min, Normalise.parentConnectionPointOpeningAngle_max) 

In [ ]:
def makeUnique(maxEventID, shift, tree_eventID) :
    for eventID in range(maxEventID + 1) :
        instances = np.where(tree_eventID == eventID)[0]

        if (len(instances) != 0) :
            groupIndex = -1
            startIndex = []
            endIndex = []
            last = -999

            for instance in instances :

                if (instance != (last + 1)) :
                    groupIndex = groupIndex + 1
                    startIndex.append(instance)
                    endIndex.append(-999)

                endIndex[groupIndex] = instance
                last = instance

            nGroups = groupIndex + 1

            for groupIndex in range(0, nGroups) :
                for particleIndex in range(startIndex[groupIndex], endIndex[groupIndex] + 1) :
                    tree_eventID[particleIndex] = tree_eventID[particleIndex] + (groupIndex * shift)

In [ ]:
##################################
# Make eventID unique -.-
##################################
nEvents = len(event_eventID)
maxEventID = np.max(event_eventID)

# Get unique eventIDs
makeUnique(maxEventID, len(event_eventID), event_eventID) 

# Build the unique eventIDs for link arrays
primaryTrack_eventID = []
primaryShower_eventID = []
trackTrack_eventID = []
trackShower_eventID = []

for iEvent in range(nEvents) :
    primaryTrack_eventID.extend([event_eventID[iEvent]] * event_nPrimaryTrackLinks[iEvent])
    primaryShower_eventID.extend([event_eventID[iEvent]] * event_nPrimaryShowerLinks[iEvent])
    trackTrack_eventID.extend([event_eventID[iEvent]] * event_nLaterTierTrackTrackLinks[iEvent])
    trackShower_eventID.extend([event_eventID[iEvent]] * event_nLaterTierTrackShowerLinks[iEvent])

In [ ]:
def GetPrimaryTrackModelInput(n_links, n_global_vars, n_link_vars, variables) :

    i_end_of_global_vars = n_global_vars
    i_target_start = variables.shape[0] - n_link_vars
    i_rest_start = n_global_vars
    i_rest_end = n_global_vars + (n_link_vars * (n_links - 1))
    
    input_0 = variables
    input_1 = np.concatenate((input_0[0:i_end_of_global_vars], input_0[i_target_start:], input_0[i_rest_start:i_rest_end]), axis=0)
    
    return input_0, input_1

def GetTrackTrackModelInput(n_links, n_global_vars, n_link_vars, variables) :

    i_end_of_global_vars = n_global_vars
    i_target_start = variables.shape[0] - n_link_vars
    i_rest_start = n_global_vars
    i_rest_end = n_global_vars + (n_link_vars * (n_links - 1))
    
    input_0 = variables
    input_1 = np.concatenate((input_0[0:i_end_of_global_vars], input_0[i_target_start:], input_0[i_rest_start:i_rest_end]), axis=0)
    input_2 = np.concatenate((input_1[0:i_end_of_global_vars], input_1[i_target_start:], input_1[i_rest_start:i_rest_end]), axis=0)
    input_3 = np.concatenate((input_2[0:i_end_of_global_vars], input_2[i_target_start:], input_2[i_rest_start:i_rest_end]), axis=0)
    
    return input_0, input_1, input_2, input_3

def GetTrackShowerModelInput(n_links, n_global_vars, n_link_vars, variables) :

    i_end_of_global_vars = n_global_vars
    i_target_start = variables.shape[0] - n_link_vars
    i_rest_start = n_global_vars
    i_rest_end = n_global_vars + (n_link_vars * (n_links - 1))
    
    input_0 = variables
    input_1 = np.concatenate((input_0[0:i_end_of_global_vars], input_0[i_target_start:], input_0[i_rest_start:i_rest_end]), axis=0)

    return input_0, input_1

In [ ]:
iTrack = 0
iShower = 0
iTrackTrack = 0
iTrackShower = 0

primary_trueGen = []
primary_trueParentID = []
primary_trackScore = []
primary_particleID = []
primary_score = []
primary_upstreamVertexX = []
primary_upstreamVertexY = []
primary_upstreamVertexZ = []
primary_downstreamVertexX = []
primary_downstreamVertexY = []
primary_downstreamVertexZ = []

laterTier_trueChildGen = []
laterTier_parentID = []
laterTier_childID = []
laterTier_nuVertexSep = []
laterTier_score = []
laterTier_parentNSpacepoints = []
laterTier_separation3D = []

for iEvent in range(len(event_eventID)):
    
    if (iEvent % 100 == 0) :
        print('iEvent:', str(iEvent) + '/' + str(MAX_EVENTS))
    
    if (iEvent > MAX_EVENTS) :
        break
    
    this_primary_trueGen = []    
    this_primary_trueParentID = []
    this_primary_trackScore = []
    this_primary_particleID = []
    this_primary_score = []
    this_primary_upstreamVertexX = []
    this_primary_upstreamVertexY = []
    this_primary_upstreamVertexZ = []
    this_primary_downstreamVertexX = []
    this_primary_downstreamVertexY = []
    this_primary_downstreamVertexZ = []    
    
    this_laterTier_trueChildGen = []
    this_laterTier_parentID = []
    this_laterTier_childID = []
    this_laterTier_nuVertexSep = []
    this_laterTier_score = [] 
    this_laterTier_parentNSpacepoints = []
    this_laterTier_separation3D = []
    
#     print('EventID:', event_eventID[iEvent])
#     print('nPrimaryTrackLinks:', event_nPrimaryTrackLinks[iEvent])
#     print('nPrimaryShowerLinks:', event_nPrimaryShowerLinks[iEvent])
#     print('nLaterTierTrackLinks:', event_nLaterTierTrackTrackLinks[iEvent])
#     print('nLaterTierShowerLinks:', event_nLaterTierTrackShowerLinks[iEvent])

    ######################################
    # Track particles
    ######################################    
    while ((iTrack < len(primaryTrack_eventID)) and (primaryTrack_eventID[iTrack] == event_eventID[iEvent])) :

        variables = [primaryTrack_nSpacepoints[iTrack]]
        
        for i in [0, 1] :
            variables.extend([primaryTrack_nuVertexSeparation[iTrack + i],
                              primaryTrack_startRegionNHits[iTrack + i],
                              primaryTrack_startRegionNParticles[iTrack + i],
                              primaryTrack_dca[iTrack + i],
                              primaryTrack_connectionExtrapDistance[iTrack + i],
                              primaryTrack_isPOIClosestToNu[iTrack + i],
                              primaryTrack_parentConnectionDistance[iTrack + i],
                              primaryTrack_childConnectionDistance[iTrack + i]])  
        
        input_0, input_1 = GetPrimaryTrackModelInput(2, 1, 8, np.array(variables))        
        predict_0 = primary_track_branch_model(torch.tensor(input_0).reshape(1, -1))
        predict_1 = primary_track_branch_model(torch.tensor(input_1).reshape(1, -1))
        classifierInput = torch.concatenate((predict_0, predict_1), axis=1)
        predict = primary_track_classifier_model(classifierInput).item()
        
        if (primaryTrack_particleID[iTrack] != primaryTrack_particleID[iTrack + 1]) :
            print('fuck')
            raise
        
        this_primary_trueGen.append(primaryTrack_trueVisibleGeneration[iTrack])
        this_primary_trueParentID.append(primaryTrack_trueVisibleParentID[iTrack])
        this_primary_trackScore.append(1.0)
        this_primary_particleID.append(primaryTrack_particleID[iTrack])
        this_primary_score.append(predict)
        this_primary_upstreamVertexX.append(primaryTrack_upstreamVertexX[iTrack])
        this_primary_upstreamVertexY.append(primaryTrack_upstreamVertexY[iTrack])
        this_primary_upstreamVertexZ.append(primaryTrack_upstreamVertexZ[iTrack])
        this_primary_downstreamVertexX.append(primaryTrack_downstreamVertexX[iTrack])
        this_primary_downstreamVertexY.append(primaryTrack_downstreamVertexY[iTrack])
        this_primary_downstreamVertexZ.append(primaryTrack_downstreamVertexZ[iTrack])

        iTrack = iTrack + 2
        
    ######################################
    # Shower particles
    ######################################         
    while ((iShower < len(primaryShower_eventID)) and (primaryShower_eventID[iShower] == event_eventID[iEvent])) :

        variables = [primaryShower_nSpacepoints[iShower], 
             primaryShower_nuVertexSeparation[iShower],
             primaryShower_startRegionNHits[iShower],
             primaryShower_startRegionNParticles[iShower],
             primaryShower_dca[iShower],
             primaryShower_connectionExtrapDistance[iShower],
             primaryShower_isPOIClosestToNu[iShower],
             primaryShower_parentConnectionDistance[iShower],
             primaryShower_childConnectionDistance[iShower]]  
        
        classifierInput = torch.tensor(variables).reshape(1, -1)
        predict = primary_shower_classifier_model(classifierInput).item() 
        
        this_primary_trueGen.append(primaryShower_trueVisibleGeneration[iShower])
        this_primary_trueParentID.append(primaryShower_trueVisibleParentID[iShower])
        this_primary_trackScore.append(0.0)
        this_primary_particleID.append(primaryShower_particleID[iShower])
        this_primary_score.append(predict)
        this_primary_upstreamVertexX.append(primaryShower_upstreamVertexX[iShower])
        this_primary_upstreamVertexY.append(primaryShower_upstreamVertexY[iShower])
        this_primary_upstreamVertexZ.append(primaryShower_upstreamVertexZ[iShower])
        this_primary_downstreamVertexX.append(primaryShower_downstreamVertexX[iShower])
        this_primary_downstreamVertexY.append(primaryShower_downstreamVertexY[iShower])
        this_primary_downstreamVertexZ.append(primaryShower_downstreamVertexZ[iShower])        
        
        iShower = iShower + 1
                
    ######################################
    # track-track particles
    ######################################         
    while ((iTrackTrack < len(trackTrack_eventID)) and (trackTrack_eventID[iTrackTrack] == event_eventID[iEvent])) :

        variables = [trackTrack_parentTrackScore[iTrackTrack], \
                     trackTrack_childTrackScore[iTrackTrack], \
                     trackTrack_parentNSpacepoints[iTrackTrack], \
                     trackTrack_childNSpacepoints[iTrackTrack], \
                     trackTrack_separation3D[iTrackTrack]]
        
        for i in [0, 1, 2, 3] :                
            variables.extend([trackTrack_parentNuVertexSep[iTrackTrack + i], \
                              trackTrack_childNuVertexSep[iTrackTrack + i], \
                              trackTrack_parentEndRegionNHits[iTrackTrack + i], \
                              trackTrack_parentEndRegionNParticles[iTrackTrack + i], \
                              trackTrack_parentEndRegionRToWall[iTrackTrack + i], \
                              trackTrack_vertexSeparation[iTrackTrack + i], \
                              trackTrack_doesChildConnect[iTrackTrack + i], \
                              trackTrack_overshootStartDCA[iTrackTrack + i], \
                              trackTrack_overshootStartL[iTrackTrack + i], \
                              trackTrack_overshootEndDCA[iTrackTrack + i], \
                              trackTrack_overshootEndL[iTrackTrack + i], \
                              trackTrack_childConnectionDCA[iTrackTrack + i], \
                              trackTrack_childConnectionExtrapDistance[iTrackTrack + i], \
                              trackTrack_childConnectionLRatio[iTrackTrack + i], \
                              trackTrack_parentConnectionPointNUpstreamHits[iTrackTrack + i], \
                              trackTrack_parentConnectionPointNDownstreamHits[iTrackTrack + i], \
                              trackTrack_parentConnectionPointNHitRatio[iTrackTrack + i], \
                              trackTrack_parentConnectionPointEigenValueRatio[iTrackTrack + i], \
                              trackTrack_parentConnectionPointOpeningAngle[iTrackTrack + i], \
                              trackTrack_isParentPOIClosestToNu[iTrackTrack + i], \
                              trackTrack_isChildPOIClosestToNu[iTrackTrack + i]])  
            
        input_0, input_1, input_2, input_3 = GetTrackTrackModelInput(4, 5, 21, np.array(variables))        
        predict_0 = track_track_branch_model(torch.tensor(input_0).reshape(1, -1))
        predict_1 = track_track_branch_model(torch.tensor(input_1).reshape(1, -1))
        predict_2 = track_track_branch_model(torch.tensor(input_2).reshape(1, -1))
        predict_3 = track_track_branch_model(torch.tensor(input_3).reshape(1, -1))
        classifierInput = torch.concatenate((predict_0, predict_1, predict_2, predict_3), axis=1)
        predict = track_track_classifier_model(classifierInput).item()
        
        for i in [1, 2, 3] :
            if ((trackTrack_parentID[iTrackTrack] != trackTrack_parentID[iTrackTrack + i]) or 
                (trackTrack_childID[iTrackTrack] != trackTrack_childID[iTrackTrack + i])) :
                    print('fuck')
                    raise        
        
        minNuVertexSep = 99999
        
        for i in [0, 1, 2, 3] :
            minNuVertexSep = min(minNuVertexSep, trackTrack_childNuVertexSep_notNorm[iTrackTrack + i])
        
        this_laterTier_nuVertexSep.append(minNuVertexSep)
        this_laterTier_trueChildGen.append(trackTrack_trueChildGen[iTrackTrack])        
        this_laterTier_parentID.append(trackTrack_parentID[iTrackTrack])
        this_laterTier_childID.append(trackTrack_childID[iTrackTrack])
        this_laterTier_score.append(predict)
        this_laterTier_parentNSpacepoints.append(trackTrack_parentNSpacepoints_notNorm[iTrackTrack])
        this_laterTier_separation3D.append(trackTrack_separation3D_notNorm[iTrackTrack])

        iTrackTrack = iTrackTrack + 4
        
    ######################################
    # track-shower particles
    ######################################         
    while ((iTrackShower < len(trackShower_eventID)) and (trackShower_eventID[iTrackShower] == event_eventID[iEvent])) :

        variables = [trackShower_parentTrackScore[iTrackShower], \
                     trackShower_childTrackScore[iTrackShower], \
                     trackShower_parentNSpacepoints[iTrackShower], \
                     trackShower_childNSpacepoints[iTrackShower], \
                     trackShower_separation3D[iTrackShower]]
        
        for i in [0, 1] :
            variables.extend([trackShower_parentNuVertexSep[iTrackShower + i], \
                              trackShower_childNuVertexSep[iTrackShower + i], \
                              trackShower_parentEndRegionNHits[iTrackShower + i], \
                              trackShower_parentEndRegionNParticles[iTrackShower + i], \
                              trackShower_parentEndRegionRToWall[iTrackShower + i], \
                              trackShower_vertexSeparation[iTrackShower + i], \
                              trackShower_doesChildConnect[iTrackShower + i], \
                              trackShower_overshootStartDCA[iTrackShower + i], \
                              trackShower_overshootStartL[iTrackShower + i], \
                              trackShower_overshootEndDCA[iTrackShower + i], \
                              trackShower_overshootEndL[iTrackShower + i], \
                              trackShower_childConnectionDCA[iTrackShower + i], \
                              trackShower_childConnectionExtrapDistance[iTrackShower + i], \
                              trackShower_childConnectionLRatio[iTrackShower + i], \
                              trackShower_parentConnectionPointNUpstreamHits[iTrackShower + i], \
                              trackShower_parentConnectionPointNDownstreamHits[iTrackShower + i], \
                              trackShower_parentConnectionPointNHitRatio[iTrackShower + i], \
                              trackShower_parentConnectionPointEigenValueRatio[iTrackShower + i], \
                              trackShower_parentConnectionPointOpeningAngle[iTrackShower + i], \
                              trackShower_isParentPOIClosestToNu[iTrackShower + i], \
                              trackShower_isChildPOIClosestToNu[iTrackShower + i]])       
            
        input_0, input_1 = GetTrackShowerModelInput(2, 5, 21, np.array(variables))        
        predict_0 = track_shower_branch_model(torch.tensor(input_0).reshape(1, -1))
        predict_1 = track_shower_branch_model(torch.tensor(input_1).reshape(1, -1))
        classifierInput = torch.concatenate((predict_0, predict_1), axis=1)
        predict = track_shower_classifier_model(classifierInput).item()
        
        if ((trackShower_parentID[iTrackShower] != trackShower_parentID[iTrackShower + 1]) or 
            (trackShower_childID[iTrackShower] != trackShower_childID[iTrackShower + 1])) :
                print('fuck')
                raise        
        
        minNuVertexSep = min(trackShower_childNuVertexSep_notNorm[iTrackShower], trackShower_childNuVertexSep_notNorm[iTrackShower + 1])
        
        this_laterTier_nuVertexSep.append(minNuVertexSep)
        this_laterTier_trueChildGen.append(trackShower_trueChildGen[iTrackShower])
        this_laterTier_parentID.append(trackShower_parentID[iTrackShower])
        this_laterTier_childID.append(trackShower_childID[iTrackShower])
        this_laterTier_score.append(predict)
        this_laterTier_parentNSpacepoints.append(trackShower_parentNSpacepoints_notNorm[iTrackShower])
        this_laterTier_separation3D.append(trackShower_separation3D_notNorm[iTrackShower])        

        iTrackShower = iTrackShower + 2      
            
    ######################################
    # Add to event arrays
    ######################################             
    primary_trueGen.append(this_primary_trueGen)
    primary_trueParentID.append(this_primary_trueParentID)
    primary_particleID.append(this_primary_particleID)
    primary_trackScore.append(this_primary_trackScore)
    primary_score.append(this_primary_score)
    primary_upstreamVertexX.append(this_primary_upstreamVertexX)
    primary_upstreamVertexY.append(this_primary_upstreamVertexY)
    primary_upstreamVertexZ.append(this_primary_upstreamVertexZ)
    primary_downstreamVertexX.append(this_primary_downstreamVertexX)
    primary_downstreamVertexY.append(this_primary_downstreamVertexY)
    primary_downstreamVertexZ.append(this_primary_downstreamVertexZ)
    
    laterTier_trueChildGen.append(this_laterTier_trueChildGen)
    laterTier_parentID.append(this_laterTier_parentID)
    laterTier_childID.append(this_laterTier_childID)
    laterTier_nuVertexSep.append(this_laterTier_nuVertexSep)
    laterTier_score.append(this_laterTier_score)    
    laterTier_parentNSpacepoints.append(this_laterTier_parentNSpacepoints)
    laterTier_separation3D.append(this_laterTier_separation3D)

                
# print('primary_trueGen:', primary_trueGen)    
# print('primarytrueParentID:', primary_trueParentID)
# print('primary_particleID:', primary_particleID)
# print('primary_trackScore:', primary_trackScore)
# print('primary_score:', primary_score) 

# print('laterTier_trueChildGen:', laterTier_trueChildGen)
# print('laterTier_parentID:', laterTier_parentID)
# print('laterTier_childID:', laterTier_childID)
# print('laterTier_nuVertexSep:', laterTier_nuVertexSep)
# print('laterTier_score:', laterTier_score)
# print('laterTier_parentNSpacepoints:', laterTier_parentNSpacepoints)
# print('laterTier_separation3D:', laterTier_separation3D)

In [ ]:
####################
# Create file mask
####################

# Primary links
primary_link_max_length = Padding.get_max_length(primary_particleID)
primary_link_mask_main = [Padding.create_mask(entry, primary_link_max_length) for entry in primary_particleID]
primary_link_mask_main = np.array(primary_link_mask_main)

print('primary_link_max_length:', primary_link_max_length)

# Higher links
laterTier_link_max_length = Padding.get_max_length(laterTier_parentID)
laterTier_link_mask_main = [Padding.create_mask(entry, laterTier_link_max_length) for entry in laterTier_parentID]
laterTier_link_mask_main = np.array(laterTier_link_mask_main)

print('laterTier_link_max_length:', laterTier_link_max_length)

In [ ]:
#############################
# Get primary link stuff primary_link_max_length
#############################
primary_particleID_main = Padding.process_array(primary_particleID, primary_link_max_length)
primary_trueGen_main = Padding.process_array(primary_trueGen, primary_link_max_length)
primary_trueParentID_main = Padding.process_array(primary_trueParentID, primary_link_max_length)
primary_trackScore_main = Padding.process_array(primary_trackScore, primary_link_max_length)
primary_score_main = Padding.process_array(primary_score, primary_link_max_length)
primary_upstreamVertexX_main = Padding.process_array(primary_upstreamVertexX, primary_link_max_length)
primary_upstreamVertexY_main = Padding.process_array(primary_upstreamVertexY, primary_link_max_length)
primary_upstreamVertexZ_main = Padding.process_array(primary_upstreamVertexZ, primary_link_max_length)
primary_downstreamVertexX_main = Padding.process_array(primary_downstreamVertexX, primary_link_max_length)
primary_downstreamVertexY_main = Padding.process_array(primary_downstreamVertexY, primary_link_max_length)
primary_downstreamVertexZ_main = Padding.process_array(primary_downstreamVertexZ, primary_link_max_length)

laterTier_trueChildGen_main = Padding.process_array(laterTier_trueChildGen, laterTier_link_max_length)
laterTier_parentID_main = Padding.process_array(laterTier_parentID, laterTier_link_max_length)
laterTier_childID_main = Padding.process_array(laterTier_childID, laterTier_link_max_length)
laterTier_nuVertexSep_main = Padding.process_array(laterTier_nuVertexSep, laterTier_link_max_length)
laterTier_score_main = Padding.process_array(laterTier_score, laterTier_link_max_length)
laterTier_parentNSpacepoints_main = Padding.process_array(laterTier_parentNSpacepoints, laterTier_link_max_length)
laterTier_separation3D_main = Padding.process_array(laterTier_separation3D, laterTier_link_max_length)

In [ ]:
np.savez(outputFile, \
         primary_link_mask_main = primary_link_mask_main, \
         primary_particleID_main = primary_particleID_main, \
         primary_trueGen_main = primary_trueGen_main, \
         primary_trueParentID_main = primary_trueParentID_main, \
         primary_trackScore_main = primary_trackScore_main, \
         primary_score_main = primary_score_main, \
         primary_upstreamVertexX_main = primary_upstreamVertexX_main, \
         primary_upstreamVertexY_main = primary_upstreamVertexY_main, \
         primary_upstreamVertexZ_main = primary_upstreamVertexZ_main, \
         primary_downstreamVertexX_main = primary_downstreamVertexX_main, \
         primary_downstreamVertexY_main = primary_downstreamVertexY_main, \
         primary_downstreamVertexZ_main = primary_downstreamVertexZ_main, \
         laterTier_link_mask_main = laterTier_link_mask_main, \
         laterTier_trueChildGen_main = laterTier_trueChildGen_main, \
         laterTier_parentID_main = laterTier_parentID_main, \
         laterTier_childID_main = laterTier_childID_main, \
         laterTier_nuVertexSep_main = laterTier_nuVertexSep_main, \
         laterTier_score_main = laterTier_score_main, \
         laterTier_parentNSpacepoints_main = laterTier_parentNSpacepoints_main, \
         laterTier_separation3D_main = laterTier_separation3D_main)